In [12]:
import pandas as pd

df = pd.read_csv("homepage_data.csv")
df = df[~df['Personal Website'].str.endswith('.sg').astype(bool)]
df.head()

,Name,Personal Website,About Us Text
0,Albert Chi-Shing CHUNG,https://cse.hkust.edu.hk/admin/people/faculty/...,Albert Chung is a Full Professor at the Depart...
1,Andrew Wing-On POON,https://eeawpoon.people.ust.hk/,Professor Poon received his BA degree from the...
2,Antoni Bert CHAN,https://www.cs.cityu.edu.hk/~abchan/,Dr. Antoni Chan is a Professor at the City Uni...
3,Brian Kan-Wing MAK,https://cse.hkust.edu.hk/admin/people/faculty/...,Dr. Mak is currently an Associate Professor in...
4,CHAN Chee Yong,https://sgps.cuhk.edu.hk/profile/professor-cha...,Professor Chee Hon Chan is an Assistant Profes...


In [13]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
from markdownify import markdownify as md
import json

load_dotenv(".env")

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

def process_homepage_text(text: str):
    response = client.chat.completions.create(
        messages=[{
            "role": "system",
            "content": """Extract the following qualities, with justification in a JSON formatted output. 
Try to infer these from the way the author writes about themselves. JSON output format: 
{
    "friendliness-justification": "Explain your friendliness quote with citations",
    "friendliness": "Scale of 1-10, how much they exhibit traits of cameraderie, teamwork and collaboration. 10 being heavy-emphasis on friendliness.",
    "we-vs-i-justification": "Explain your we-vs-i quote with citations",
    "we-vs-i": "Scale of 1-10, how much they talk about themselves vs their collaborators. 10 being heavy-emphasis on we.",
    "risk-averse-justification": "Explain your risk-averse quote with citations",
    "risk-averse": "Scale of 1-10, how they feel about taking risks. 10 being very risk-averse.",
    "career-orientation-justification": "Explain your career-orientation quote with citations",
    "career-orientation": "How much a person cares about their career. 10 being very career-oriented.",
}"""
        }, {
            "role": "user",
            "content": text
        }],
        response_format={
            "type": "json_object"
        },
        model="gpt-4o"
    )
    response = json.loads(response.choices[0].message.content)
    return response


In [14]:
from rich import print
from tqdm import tqdm

# TODO: append university profile w/ personal website text
# TODO: make sure that results aren't correlated w/ more text (more personable because they wrote more) 

researcher_profile = []

for index, row in tqdm(df.iterrows()):
    text = row['Personal Website']
    response = process_homepage_text(text)
    response["name"] = row['Name']
    researcher_profile.append(response)
    

78it [04:08,  3.19s/it]


In [15]:
with open("homepage_data_analyzed.json", "w+") as f:
    json.dump(researcher_profile, f)

In [8]:
# TODO: save to 0721-month-analysis-data, save to new file